In [3]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold, RepeatedStratifiedKFold, cross_val_predict
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, recall_score, roc_auc_score
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, RandomForestClassifier

import matplotlib.pyplot as plt

import time
from sklearn.linear_model import Lasso
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,train_test_split, KFold, StratifiedKFold, GridSearchCV, ParameterGrid, \
RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score, mean_squared_error
from skopt import BayesSearchCV


import os
os.environ["OMP_NUM_THREADS"] = "1"

In [4]:
train = pd.read_csv('train_classification.csv')
test = pd.read_csv('test_classification.csv')

In [5]:

## turned y value numeric
most_frequent_value = train['host_is_superhost'].mode()[0]
train['host_is_superhost'].fillna(most_frequent_value, inplace=True)

train['host_is_superhost'] = train['host_is_superhost'].replace({'f': 0, 't': 1})

## turned min_nights numeric
train.minimum_minimum_nights = train.minimum_minimum_nights.astype(float)
test.minimum_minimum_nights = test.minimum_minimum_nights.astype(float)

## filled in any missing values for bed
train.beds = train.beds.bfill()
test.beds = test.beds.bfill()

## tip from professor


##important value -- used the average
train['review_scores_value'] = train['review_scores_value'].fillna(train.review_scores_value.mean())
test['review_scores_value'] = test['review_scores_value'].fillna(test.review_scores_value.mean())




most_frequent_value = train['review_scores_location'].mode()[0]
train['review_scores_location'].fillna(most_frequent_value, inplace=True)

most_frequent_value = test['review_scores_location'].mode()[0]
test['review_scores_location'].fillna(most_frequent_value, inplace=True)


most_frequent_value = train['host_response_rate'].mode()[0]
train['host_response_rate'].fillna(most_frequent_value, inplace=True)


most_frequent_value = test['host_response_rate'].mode()[0]
test['host_response_rate'].fillna(most_frequent_value, inplace=True)


most_frequent_valuee = test['host_response_time'].mode()[0]
test['host_response_time'].fillna(most_frequent_valuee, inplace=True)

most_frequent_valuee = train['host_response_time'].mode()[0]
train['host_response_time'].fillna(most_frequent_valuee, inplace=True)


## tuened response rate from percentage to rate
train.host_response_rate = train.host_response_rate.str.replace('%', '').str.replace(',' , '').astype(float)
test.host_response_rate = test.host_response_rate.str.replace('%', '').str.replace(',' , '').astype(float)






## filled in any missing values with median as data was skewed and numeric
train.host_response_rate = train.host_response_rate.fillna(train.host_response_rate.median())
test.host_response_rate = test.host_response_rate.fillna(test.host_response_rate.median())



train.host_acceptance_rate = train.host_acceptance_rate.fillna(train.host_response_rate.median())
test.host_acceptance_rate = test.host_acceptance_rate.fillna(test.host_response_rate.median())

train.review_scores_rating = train.review_scores_rating.fillna(train.review_scores_rating.median())
test.review_scores_rating = test.review_scores_rating.fillna(test.review_scores_rating.median())

train.review_scores_accuracy = train.review_scores_accuracy.fillna(train.review_scores_accuracy.median())
test.review_scores_accuracy = test.review_scores_accuracy.fillna(test.review_scores_accuracy.median())


train.review_scores_cleanliness = train.review_scores_cleanliness.fillna(train.review_scores_cleanliness.median())
test.review_scores_cleanliness = test.review_scores_cleanliness.fillna(test.review_scores_cleanliness.median())

train.review_scores_checkin = train.review_scores_checkin.fillna(train.review_scores_checkin.median())
test.review_scores_checkin = test.review_scores_checkin.fillna(test.review_scores_checkin.median())

train.review_scores_communication = train.review_scores_communication.fillna(train.review_scores_communication.median())
test.review_scores_communication = test.review_scores_communication.fillna(test.review_scores_communication.median())


#filled in any missing values with at least one 
test['bathrooms_text'] = test['bathrooms_text'].fillna('1 bath')

train['bathrooms_text'] = train['bathrooms_text'].fillna('1 bath')

train['year'] = pd.DatetimeIndex(train['host_since']).year
test['year'] = pd.DatetimeIndex(test['host_since']).year


import math

def log_review(x):
    if x == 0:
        return 0
    else:
        return math.log(x)

train['reviews_log'] = train.number_of_reviews.apply(log_review)
test['reviews_log'] = test.number_of_reviews.apply(log_review)


## began dealing with bathrooms_text, distinguished those with shared bathrooms 
train['shared_bathroom'] = train['bathrooms_text'].str.contains('shared', case=False, na=False).astype(int)
test['shared_bathroom'] = test['bathrooms_text'].str.contains('shared', case=False, na=False).astype(int)



train['instant_bookable'] = train['instant_bookable'].map({'f': 0, 't': 1})
test['instant_bookable'] = test['instant_bookable'].map({'f': 0, 't': 1})

test['has_availability'] = test['has_availability'].map({'f': 0, 't': 1})
train['has_availability'] = train['has_availability'].map({'f': 0, 't': 1})


test['host_identity_verified'] = test['host_identity_verified'].map({'f': 0, 't': 1})
train['host_identity_verified'] = train['host_identity_verified'].map({'f': 0, 't': 1})




test['host_has_profile_pic'] = test['host_has_profile_pic'].map({'f': 0, 't': 1})
train['host_has_profile_pic'] = train['host_has_profile_pic'].map({'f': 0, 't': 1})


train.host_acceptance_rate = train.host_acceptance_rate.str.replace('%', '').str.replace(',' , '').astype(float)
test.host_acceptance_rate = test.host_acceptance_rate.str.replace('%', '').str.replace(',' , '').astype(float)


train.host_acceptance_rate = train.host_acceptance_rate.fillna(train.host_acceptance_rate.median())
test.host_acceptance_rate = test.host_acceptance_rate.fillna(test.host_acceptance_rate.median())



test['reviews_per_month'] = test['reviews_per_month'].fillna("0")

train['reviews_per_month'] = train['reviews_per_month'].fillna("0")



test['host_neighbourhood'] = test['host_neighbourhood'].fillna("Other")

train['host_neighbourhood'] = train['host_neighbourhood'].fillna("Other")

test['host_location'] = test['host_location'].fillna("Other")

train['host_location'] = train['host_location'].fillna("Other")


/tmp/ipykernel_234402/143400930.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['host_is_superhost'].fillna(most_frequent_value, inplace=True)
/tmp/ipykernel_234402/143400930.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['host_is_superhost'] = train['host_is_superhost'].replace

In [6]:

top_n = 3  
top_categories = train['host_location'].value_counts().index[:top_n]

def bin_host_location(location):
    if location in top_categories:
        return location
    else:
        return 'Other'

train['host_location_binned'] = train['host_location'].apply(bin_host_location)
test['host_location_binned'] = test['host_location'].apply(bin_host_location)




top_n = 3  
top_categories = train['host_neighbourhood'].value_counts().index[:top_n]

def bin_host_neighbourhood(neighbourhood):
    if neighbourhood in top_categories:
        return neighbourhood
    else:
        return 'Other'

train['host_neighbourhood_binned'] = train['host_neighbourhood'].apply(bin_host_neighbourhood)
test['host_neighbourhood_binned'] = test['host_neighbourhood'].apply(bin_host_neighbourhood)



top_n = 3 
top_categories = train['property_type'].value_counts().index[:top_n]

def bin_property_type(property):
    if property in top_categories:
        return property
    else:
        return 'Other'

train['property_binned'] = train['property_type'].apply(bin_property_type)
test['property_binned'] = test['property_type'].apply(bin_property_type)



top_n = 3 
top_categories = train['neighbourhood_cleansed'].value_counts().index[:top_n]

def bin_neighbourhood_cleansed(neighbourhood):
    if neighbourhood in top_categories:
        return neighbourhood
    else:
        return 'Other'

train['neighbourhood_cleansed_binned'] = train['neighbourhood_cleansed'].apply(bin_neighbourhood_cleansed)
test['neighbourhood_cleansed_binned'] = test['neighbourhood_cleansed'].apply(bin_neighbourhood_cleansed)

In [7]:
predictors = ['host_response_time', 'host_response_rate',
       'host_acceptance_rate',  
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed_binned', 'latitude', 'longitude', 'property_binned',
       'room_type', 'accommodates', 'beds', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month','year', 'reviews_log',
             'host_neighbourhood_binned', 'host_location_binned']



X_train = train[predictors]
y_train = train['host_is_superhost']
X_test = test[predictors]

In [8]:
X_test.host_verifications

X_test['host_verifications'] = X_test['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")

X_train.host_verifications

X_train['host_verifications'] = X_train['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")


relevant_columns = X_train.loc[:,["host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
train_clean = pd.get_dummies(relevant_columns, drop_first = True)
train_clean
X_train_total = pd.concat([X_train.drop(relevant_columns.columns, axis=1), train_clean], axis=1)

relevant_columns = X_test.loc[:,[ "host_has_profile_pic", 
                 "host_identity_verified" , 
                                  'neighbourhood_cleansed_binned' , 'property_binned', 'room_type', 'host_neighbourhood_binned', 'host_location_binned', 'host_response_time', 'host_verifications']]
test_clean = pd.get_dummies(relevant_columns, drop_first = True)
test_clean
X_test_total = pd.concat([X_test.drop(relevant_columns.columns, axis=1), test_clean], axis=1)

X_train_total.reviews_per_month = X_train_total.reviews_per_month.astype(float)
X_test_total.reviews_per_month = X_test_total.reviews_per_month.astype(float)

/tmp/ipykernel_234402/3117073857.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['host_verifications'] = X_test['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")
/tmp/ipykernel_234402/3117073857.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['host_verifications'] = X_train['host_verifications'].str.replace("[", "").str.replace("]", "").str.replace("'", "")


In [9]:
scaler = StandardScaler()


scaler.fit(X_train_total)

Xstd = scaler.transform(X_train_total)

X_test_scaled = scaler.transform(X_test_total)


In [10]:
## 78640


cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)

# The rest is the same as KFold
model = KNeighborsClassifier()

grid = { 'leaf_size': [5, 10,15], 'p': [1], 'n_neighbors': [3,4,6]}



gcv = GridSearchCV(model, grid, cv = cv,  n_jobs= int(os.getenv("SLURM_NPROCS", 1)), verbose =1)


gcv.fit(Xstd, y_train)

gcv.best_estimator_
gcv.best_params_

y_pred = gcv.predict(X_test_scaled)
gcv.best_params_




Fitting 50 folds for each of 9 candidates, totalling 450 fits


{'leaf_size': 5, 'n_neighbors': 6, 'p': 1}

In [11]:
model = KNeighborsClassifier(n_neighbors = 6, p = 1, leaf_size = 5)

model.fit(Xstd, y_train)
y_pred = model.predict(X_test_scaled)


output = pd.DataFrame({'id':test.id, 'predicted':y_pred})
output


output.to_csv("submission.csv", index= False)
## 0.8110